In [67]:
from part2_house_value_regression import Regressor
from sklearn import model_selection
import pandas as pd
import collections
import math
import random

Generate first folds

In [68]:
output_label = "median_house_value"
data = pd.read_csv("housing.csv") 
x_train = data.loc[:, data.columns != output_label]
y_train = data.loc[:, [output_label]]
def genModel(hyperparam, x, y):
    xTrain, xValidation, yTrain, yValidation = model_selection.train_test_split(x, y, test_size=0.1)
    model = model_selection.GridSearchCV(
        estimator = Regressor(),
        param_grid = hyperparam,
        scoring="neg_root_mean_squared_error", #Scoring metric means lower is better
        cv=3,
        verbose=4,
        n_jobs=1,
        return_train_score = True
        )
    model.fit(xTrain, yTrain, xValidation=xValidation, yValidation=yValidation)
    print("model complete")
    return model
    
#This function will find the top two paramters and create a range between them
def getTopTwo(inputList):
    paramHeaders = {"nb_epoch" : 0, "learningRate" : 1, "neuronArchitecture" : 2, "batchSize" : 3}
    params = [[] for i in range(len(paramHeaders))]
    paramMode = dict()
    #Convert list of dictionaries to list per parameter
    for description in inputList:
        for key, value in description.items():
            params[paramHeaders[key]].append(tuple(value) if isinstance(value, list) else value)
    #Invert the dictionary
    paramInverted = {value: key for key, value in paramHeaders.items()}
    #Obtain the two most common items
    for index, value in enumerate(params):
        mode = collections.Counter(params[index]).most_common(2)
        paramMode[paramInverted[index]] = [i[0] for i in mode]
    return paramMode


In [71]:
# hyperparam = {
#         "nb_epoch" : [150], 
#         "learningRate" : [0.001, 0.01, 0.1], 
#         "neuronArchitecture" : [[9], [9,9], [9,9,9]], 
#         "batchSize" : [64, 128, 256, 512],
#         }
hyperparam = {
        'nb_epoch': [150], 
        'learningRate': [0.03859925930841978, 0.07261032139183789, 0.018281914050446196, 0.026122808062702445], 
        'neuronArchitecture': [[11, 10], [10, 8], [12, 10], [10, 9]], 
        'batchSize': [81, 61, 123, 88]
        }
model = genModel(hyperparam, x_train, y_train)

Fitting 3 folds for each of 64 candidates, totalling 192 fits
0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-Reached epoch cycle: 30 with error: 72192.151364384
[CV 1/3] END batchSize=81, learningRate=0.03859925930841978, nb_epoch=150, neuronArchitecture=[11, 10];, score=(train=-70354.137, test=-71767.533) total time=   6.2s
0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-Reached epoch cycle: 39 with error: 70414.23044000754
[CV 2/3] END batchSize=81, learningRate=0.03859925930841978, nb_epoch=150, neuronArchitecture=[11, 10];, score=(train=-68285.154, test=-69312.106) total time=   7.9s
0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-Reached epoch cycle: 28 with error: 71983.87934412174
[CV 3/3] END batchSize=81, learningRate=0.03859925930841978, nb_epoch=150, neuronArchitecture=[11, 10];, score=(train=-71061.372, test=-69089.786) total time=   5.7s
0-1-2-3-4-5-6-7-8-9

In [72]:

results = pd.DataFrame(model.cv_results_)
print(results)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        6.648609      0.939340         0.017287        0.002236   
1        7.015792      0.472853         0.017136        0.001798   
2        7.034293      0.832717         0.017017        0.000800   
3        6.798777      0.623879         0.017873        0.000980   
4        8.304780      2.249763         0.016003        0.000819   
..            ...           ...              ...             ...   
59       9.998491      0.548352         0.020549        0.000231   
60       7.891523      0.809188         0.021552        0.001043   
61       8.574922      1.079544         0.019391        0.002795   
62       8.638156      1.058011         0.020131        0.001039   
63       8.369054      1.023424         0.021019        0.001543   

   param_batchSize param_learningRate param_nb_epoch param_neuronArchitecture  \
0               81           0.038599            150                 [11, 10]   
1               81   

In [73]:
bestPerformer = results["mean_test_score"].max()
print(bestPerformer)
bestParams = model.best_params_
print(bestParams)

-67870.703338204
{'batchSize': 81, 'learningRate': 0.07261032139183789, 'nb_epoch': 150, 'neuronArchitecture': [11, 10]}


In [74]:
print(results)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        6.648609      0.939340         0.017287        0.002236   
1        7.015792      0.472853         0.017136        0.001798   
2        7.034293      0.832717         0.017017        0.000800   
3        6.798777      0.623879         0.017873        0.000980   
4        8.304780      2.249763         0.016003        0.000819   
..            ...           ...              ...             ...   
59       9.998491      0.548352         0.020549        0.000231   
60       7.891523      0.809188         0.021552        0.001043   
61       8.574922      1.079544         0.019391        0.002795   
62       8.638156      1.058011         0.020131        0.001039   
63       8.369054      1.023424         0.021019        0.001543   

   param_batchSize param_learningRate param_nb_epoch param_neuronArchitecture  \
0               81           0.038599            150                 [11, 10]   
1               81   

In [75]:
candidateThreshold = 0.05
resultsTop = results.loc[results["mean_test_score"] >= bestPerformer*(1+candidateThreshold)]
#print(resultsTop)
paramList = resultsTop["params"].values
#Now, calculate all the new hyperparameters and prepare for next round
print("Found params:", paramList)

Found params: [{'batchSize': 81, 'learningRate': 0.03859925930841978, 'nb_epoch': 150, 'neuronArchitecture': [11, 10]}
 {'batchSize': 81, 'learningRate': 0.03859925930841978, 'nb_epoch': 150, 'neuronArchitecture': [10, 8]}
 {'batchSize': 81, 'learningRate': 0.03859925930841978, 'nb_epoch': 150, 'neuronArchitecture': [12, 10]}
 {'batchSize': 81, 'learningRate': 0.03859925930841978, 'nb_epoch': 150, 'neuronArchitecture': [10, 9]}
 {'batchSize': 81, 'learningRate': 0.07261032139183789, 'nb_epoch': 150, 'neuronArchitecture': [11, 10]}
 {'batchSize': 81, 'learningRate': 0.07261032139183789, 'nb_epoch': 150, 'neuronArchitecture': [10, 8]}
 {'batchSize': 81, 'learningRate': 0.07261032139183789, 'nb_epoch': 150, 'neuronArchitecture': [12, 10]}
 {'batchSize': 81, 'learningRate': 0.07261032139183789, 'nb_epoch': 150, 'neuronArchitecture': [10, 9]}
 {'batchSize': 81, 'learningRate': 0.018281914050446196, 'nb_epoch': 150, 'neuronArchitecture': [11, 10]}
 {'batchSize': 81, 'learningRate': 0.0182819

Search for best params

In [76]:
newParams = getTopTwo(paramList)
print("Optimum params:", newParams)

Optimum params: {'nb_epoch': [150], 'learningRate': [0.03859925930841978, 0.07261032139183789], 'neuronArchitecture': [(11, 10), (10, 8)], 'batchSize': [61, 81]}


In [77]:
#Find magnitudes
#Determine the amount of layers - prefer less layers
layerCount = min([len(x) for x in newParams["neuronArchitecture"]])
#Determine the magnitude of the learning rate 
if len(newParams["learningRate"]) >= 2:
    learningMagnitude = sum([math.log(x, 10) for x in newParams["learningRate"][:2]])/2
else:
    learningMagnitude = math.log(newParams["learningRate"][0], 10)
print("Layercount:", layerCount, "Learning Magnitude:", learningMagnitude, "Learning rate approx:", 10**learningMagnitude)

Layercount: 2 Learning Magnitude: -1.2762113349911144 Learning rate approx: 0.05294057634623231


In [78]:
newhyperparam = {"nb_epoch" : None, "learningRate" : None, "neuronArchitecture" : [], "batchSize" : None}
magnitudeModifier = 0.6
neuronModifier = 3
newhyperparam["learningRate"] = [10**random.uniform(learningMagnitude-magnitudeModifier, learningMagnitude+magnitudeModifier) for _ in range(4)]
#Neuron architecture
for i in range(4):
    maxNeurons = 13
    architecture = []
    for j in range(layerCount):
        #Ensure decreasing neurons
        maxNeurons = random.randint(maxNeurons-neuronModifier, maxNeurons)
        architecture.append(maxNeurons)
    newhyperparam["neuronArchitecture"].append(architecture)
#Batchsize
if len(newParams["batchSize"]) >= 2:
    batchMagnitude = sum([math.log(x, 2) for x in newParams["batchSize"][:2]])/2
else:
    batchMagnitude = math.log(newParams["batchSize"][0], 2)
newhyperparam["batchSize"] = [int(2**random.uniform(batchMagnitude-magnitudeModifier, batchMagnitude+magnitudeModifier)) for _ in range(4)]
newhyperparam["nb_epoch"] = newParams["nb_epoch"]
print("New hyperparameters:", newhyperparam)

New hyperparameters: {'nb_epoch': [150], 'learningRate': [0.014798359493541224, 0.02289178310567298, 0.06102059645509449, 0.1648181761144886], 'neuronArchitecture': [[11, 8], [11, 10], [12, 10], [11, 11]], 'batchSize': [98, 46, 80, 61]}
